# 📌 ACTIVIDAD 5: REGRESIÓN A PARTIR DE FOTOGRAFÍAS.

## DEFINIR PROBLEMA Y RECOPILAR DATOS

Crea el notebook saa_u03_p01_a5-<tus_iniciales>.ipynb donde entregar esta actividad. Necesitamos
consensuar por votación 2 posibles problemas (lo que escoja la mayoría de la clase gana) más que
nada por obtener suficiente cantidad de datos de alguno de los problemas:

 a) Predecir la edad de una persona: Si nadie está en contra de aportar fotografías personales,
cada alumno buscará 10 fotografías suyas o de conocidos (propias, familia, amigos, …)
realizadas en diferentes edades y las etiquetará con la edad que tenía en ese momento la
persona que aparece "edad_<tus_iniciales>-<num_foto>.jpg" o "edad_<tus_iniciales>
<num_foto>.png". En el caso de descargar de Internet las imágenes o de generarlas con
aplicaciones tened cuidado porque al buscar os pueden aparecer las mismas fotografías para
diferentes edades. La cara debe cubrir casi toda la foto (sin paisaje de fondo: ajustar el borde de
abajo a la barbilla y los laterales a las orejas y el borde superior al pelo) y la persona debe estar
mirando de frente.

 b) Predecir la peligrosidad de un animal en un rango de 0 a 10: 10 significa que te puede
matar o desgraciar si te engancha y 0 que no te va a dañar (al menos en principio). En caso de
escoger esta opción cada uno buscará, procesará y aportará 10 fotografías de cabezas de
animales de todo tipo (serpientes, insectos, felinos, osos, tiburones, ovejas, gatitos, …) con el
nombre del fichero siguiendo el formato "peligo_<tus_iniciales>-<num_foto>.jpg" o bien
formato "peligo_<tus_iniciales>-<num_foto>.png".


Nota: este enfoque no tiene visos de dar buenos resultados. Lo ideal sería extraer característics de cada
fotografía (zonas de ojos, boca, nariz, orejas, dientes) creando embeddings y codificando estos rasgos a
través de deep learning y luego usarlos para realizar las predicciones, pero vamos a probar a ver que tal
nos va, al fin y al cabo es una excusa para probar regresores.

 Una vez que tengas las fotografías debes procesarlas. Te paso el siguiente código:



In [ ]:
import numpy as np
import pandas as pd
import fe
import cv2
import os

# ## Instalar opencv: pip install opencv-python

carpeta = "./"  # ## CAMBIA!! ## Ruta de carpeta donde están las imágenes
archivo_salida = "josrosrod_imagenes.csv"  # ## CAMBIA!! ## Ruta de archivo donde guardar datos como .csv
# Recorrer todas las imágenes en la carpeta
datos_procesados = []
patron = r"^\d+.*\.jpe?g$"  # Para archivos que empiezan con uno o más dígitos y terminan en .jpg o .jpeg
for nombre_archivo in os.listdir(carpeta):
    ruta_completa = os.path.join(carpeta, nombre_archivo)
    if not re.match(patron, nombre_archivo):  # Si el fichero no es imagen con dato numérico antes...
        continue
    try:
        edad = int(nombre_archivo.split("_")[0])
    except ValueError:
        print(f"No se pudo extraer la edad del nombre del archivo: {nombre_archivo}")
        continue
    # Leer la imagen en escala de grises, escalarla, guardar la procesada
    imagen = cv2.imread(ruta_completa, cv2.IMREAD_GRAYSCALE)
    if imagen is None:
        print(f"No se pudo leer: {nombre_archivo}")
        continue
    imagen_escalada = cv2.resize(imagen, (92, 112), interpolation=cv2.INTER_AREA) # Escala a 92x112
    cv2.imwrite(os.path.join(carpeta, "img_" + nombre_archivo), imagen_escalada)
    imagen_normalizada = (imagen_escalada / 255.0).astype(np.float32) # Normalizar a [0,1]
    datos_procesados.append({"edad": edad, "imagen": imagen_normalizada}) # Añadir datos a datos_procesados

# Convertir datos_procesados a DataFrame y guardar como csv
df = pd.DataFrame({"edad": [d["edad"] for d in datos_procesados],
                   "imagen": [",".join(map(str, d["imagen"].ravel())) for d in datos_procesados]})
df.to_csv(archivo_salida, index=False)
print(f"✅ Se guardaron {len(datos_procesados)} imágenes procesadas en '{archivo_salida}'")

 Lo que hace el código es definir rutas (que debes adaptar para tu uso) en las variables carpeta (ruta
relativa para alcanzar el lugar donde están las imágenes) y archivo_salida (pathname relativo que
define el archivo .csv donde se van a guardar los datos).

 Las imágenes de carpeta se transforman usando la librería opencv (quizás debas instalarla) en
información numérica de la siguiente manera: obtendremos una imagen en escala de grises de
dimensiones 92x112 píxels (ancho x alto) que se almacenan como valores float de 32 bits sin signo
entre 0 y 256 normalizados a float en el intervalo [0,1]. La columna target de cada foto será la primera
característica del dataset. Por ejemplo podemos tener ficheros como estos:


 Y al ejecutar el script obtenermos el resultado y generamos:
